In [18]:
# Libraries to help with reading and manipulating data
import pandas as pd
import numpy as np


In [19]:
file_path = "/Users/taylorbaldwin/Downloads/Credit Card Defaulter Prediction.csv"
df = pd.read_csv(file_path)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,1,20000,F,University,Married,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,Y
1,2,120000,F,University,Single,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,Y
2,3,90000,F,University,Single,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,N
3,4,50000,F,University,Married,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,N
4,5,50000,M,University,Married,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,N


In [36]:
df['PAY_0']

0        2
1       -1
2        0
3        0
4       -1
        ..
29995    0
29996   -1
29997    4
29998    1
29999    0
Name: PAY_0, Length: 29297, dtype: int64

In [27]:
# Remove rows where 'MARRIAGE' has the value '0' and 'Unknown'
values_to_remove = ['Unknown', '0']
df_filtered = df[~df['MARRIAGE'].isin(values_to_remove)]

# Remove rows where 'Education has the value 'Others' and 'Unknown'
values_to_remove1 = ['Unknown', '0']
df = df_filtered[~df_filtered['MARRIAGE'].isin(values_to_remove1)]

In [28]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default '],
      dtype='object')

In [ ]:
SEX_MAPPING = {'M': 'Male', 'F': 'Female'}
DEFAULT_MAPPING = {'N': 'Non-Defaulter', 'Y': 'Defaulter'}
PAY_STATUS_MAPPING = {
    -2: 'Account Closed',
    -1: 'Payment Delayed',
    0: 'Revolving Credit',
    1: 'Delay 1 Month',
    2: 'Delay 2 Months',
    3: 'Delay 3 Months',
    4: 'Delay 4 Months',
    5: 'Delay 5 Months',
    6: 'Delay 6 Months',
    7: 'Delay 7 Months',
    8: 'Delay 8 Months',
    9: 'Delay 9+ Months'
}

def clean_and_prepare_data(file_path):

    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

    print(f"--- Data loaded successfully with {len(df)} rows. ---")

    # === 2. RENAME COLUMNS ===
    df.rename(columns={'ID': 'Client_ID', 'PAY_0': 'PAY_1'}, inplace=True)

    if 'ID' in df.columns:
        df.drop('ID', axis=1, inplace=True)

    # === 3. RECODE CATEGORICALS ===
    print("--- Recoding categorical features ---")
    df['SEX'] = df['SEX'].map(SEX_MAPPING)
    df['default '] = df['default '].map(DEFAULT_MAPPING)

    # === Convert all numeric columns to integers ===
    print("--- Converting all numeric columns to integers ---")
    numeric_cols = df.select_dtypes(include=['number']).columns

    # Convert numeric columns safely
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # === 4. Melt sequences ===
    bill_cols = [f'BILL_AMT{i}' for i in range(1, 7)]
    pay_cols = [f'PAY_{i}' for i in range(1, 7)]
    pay_amt_cols = [f'PAY_AMT{i}' for i in range(1, 7)]

    id_vars = ['Client_ID', 'default ', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'LIMIT_BAL']

    print("--- Preparing BILL_AMT (long) ---")
    df_bill_long = pd.melt(
        df, id_vars=id_vars, value_vars=bill_cols,
        var_name='Bill_Month', value_name='Bill_Amount'
    )
    df_bill_long['Bill_Month'] = df_bill_long['Bill_Month'].str.replace("BILL_AMT", "Month ")

    print("--- Preparing PAY_STATUS (long) ---")
    df_pay_long = pd.melt(
        df, id_vars=id_vars, value_vars=pay_cols,
        var_name='Pay_Month', value_name='Pay_Status_Code'
    )
    df_pay_long['Pay_Month'] = df_pay_long['Pay_Month'].str.replace("PAY_", "Month ")
    df_pay_long['Pay_Status_Label'] = df_pay_long['Pay_Status_Code'].map(PAY_STATUS_MAPPING)

    print("--- Preparing PAY_AMT (long) ---")
    df_pay_amt_long = pd.melt(
        df, id_vars=id_vars, value_vars=pay_amt_cols,
        var_name='Pay_Amt_Month', value_name='Payment_Amount'
    )
    df_pay_amt_long['Pay_Amt_Month'] = df_pay_amt_long['Pay_Amt_Month'].str.replace("PAY_AMT", "Month ")

    # === 5. EXPORT TO DOWNLOADS FOLDER ===
    output_dir = "/Users/taylorbaldwin/Downloads/"

    df.to_csv(output_dir + "cleaned_default_data_wide.csv", index=False)
    df_bill_long.to_csv(output_dir + "cleaned_bill_amt_long.csv", index=False)
    df_pay_long.to_csv(output_dir + "cleaned_pay_status_long.csv", index=False)
    df_pay_amt_long.to_csv(output_dir + "cleaned_pay_amt_long.csv", index=False)

    print("\n--- Data Cleaning Complete! ---")
    print("Saved files to your Downloads folder:")
    print(f" - {output_dir}cleaned_default_data_wide.csv")
    print(f" - {output_dir}cleaned_bill_amt_long.csv")
    print(f" - {output_dir}cleaned_pay_status_long.csv")
    print(f" - {output_dir}cleaned_pay_amt_long.csv")

    return {
        'wide_data': df,
        'bill_data_long': df_bill_long,
        'pay_status_data_long': df_pay_long,
        'pay_amt_data_long': df_pay_amt_long
    }

In [30]:
cleaned_data = clean_and_prepare_data(file_path)

--- Data loaded successfully with 30000 rows. ---
--- Recoding categorical features ---
--- Converting all numeric columns to integers ---
--- Preparing BILL_AMT (long) ---
--- Preparing PAY_STATUS (long) ---
--- Preparing PAY_AMT (long) ---
--- Preparing PAY_STATUS (long) ---
--- Preparing PAY_AMT (long) ---

--- Data Cleaning Complete! ---
Saved files to your Downloads folder:
 - /Users/taylorbaldwin/Downloads/cleaned_default_data_wide.csv
 - /Users/taylorbaldwin/Downloads/cleaned_bill_amt_long.csv
 - /Users/taylorbaldwin/Downloads/cleaned_pay_status_long.csv
 - /Users/taylorbaldwin/Downloads/cleaned_pay_amt_long.csv

--- Data Cleaning Complete! ---
Saved files to your Downloads folder:
 - /Users/taylorbaldwin/Downloads/cleaned_default_data_wide.csv
 - /Users/taylorbaldwin/Downloads/cleaned_bill_amt_long.csv
 - /Users/taylorbaldwin/Downloads/cleaned_pay_status_long.csv
 - /Users/taylorbaldwin/Downloads/cleaned_pay_amt_long.csv


In [31]:
# Create a stratified sample of 1,000 rows preserving 'default ' distribution
if cleaned_data is None:
    raise ValueError("cleaned_data is None. Run clean_and_prepare_data first.")
df_wide = cleaned_data['wide_data']
total = len(df_wide)
print(f"Source dataset rows: {total}")
sample_n = 1000
if total <= sample_n:
    print("Source has <=1000 rows; saving full dataset as sample.")
    sample_df = df_wide.copy()
else:
    group_col = 'default '
    proportions = df_wide[group_col].value_counts(normalize=True)
    raw_counts = proportions * sample_n
    floor_counts = raw_counts.astype(int)
    remainder = raw_counts - floor_counts
    desired = floor_counts.copy()
    remaining = sample_n - desired.sum()
    if remaining > 0:
        for idx in remainder.sort_values(ascending=False).index:
            if remaining <= 0:
                break
            desired[idx] += 1
            remaining -= 1
    parts = []
    for grp, n in desired.items():
        grp_df = df_wide[df_wide[group_col] == grp]
        available = len(grp_df)
        if available >= n:
            parts.append(grp_df.sample(n=n, random_state=42))
        else:
            parts.append(grp_df.sample(n=available, random_state=42))
    sample_df = pd.concat(parts)
    if len(sample_df) < sample_n:
        needed = sample_n - len(sample_df)
        remaining_pool = df_wide.drop(sample_df.index)
        if len(remaining_pool) >= needed:
            extra = remaining_pool.sample(n=needed, random_state=42)
        else:
            extra = remaining_pool.sample(n=needed, replace=True, random_state=42)
        sample_df = pd.concat([sample_df, extra])
    sample_df = sample_df.sample(frac=1, random_state=42).reset_index(drop=True)

output_file = "/Users/taylorbaldwin/Downloads/cleaned_default_data_sample_1000.csv"
sample_df.to_csv(output_file, index=False)
print(f"Saved stratified sample (n={len(sample_df)}) to {output_file}")

sample_df.head()

Source dataset rows: 30000
Saved stratified sample (n=1000) to /Users/taylorbaldwin/Downloads/cleaned_default_data_sample_1000.csv


,Client_ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,28639,100000,Female,High School,Married,39,0,0,0,0,...,184134,70400,0,4296,4068,2037,1408,0,0,Non-Defaulter
1,14389,50000,Male,High School,Single,22,0,0,0,0,...,7694,7921,8041,2400,1123,500,500,400,500,Non-Defaulter
2,21171,170000,Male,Graduate school,Single,42,0,0,0,0,...,91719,97759,99905,4000,4000,5000,7500,4000,5000,Non-Defaulter
3,3573,20000,Female,University,Married,24,1,2,0,0,...,20298,20337,19863,0,1720,1754,893,0,400,Non-Defaulter
4,5242,140000,Female,Graduate school,Single,31,0,0,0,2,...,43051,44109,43253,1732,2278,3000,1892,0,1572,Non-Defaulter


In [32]:
# Create stratified 1,000-row samples for the three long tables (bill, pay status, pay amt)
if cleaned_data is None:
    raise ValueError("cleaned_data is None. Run clean_and_prepare_data first.")
long_tables = {
    'bill_data_long': (cleaned_data['bill_data_long'], '/Users/taylorbaldwin/Downloads/cleaned_bill_amt_sample_1000.csv'),
    'pay_status_data_long': (cleaned_data['pay_status_data_long'], '/Users/taylorbaldwin/Downloads/cleaned_pay_status_sample_1000.csv'),
    'pay_amt_data_long': (cleaned_data['pay_amt_data_long'], '/Users/taylorbaldwin/Downloads/cleaned_pay_amt_sample_1000.csv'),
}
sample_n = 1000
group_col = 'default '
for name, (df_long, out_path) in long_tables.items():
    print(f"Sampling {name} (source rows: {len(df_long)}) -> {out_path}")
    if len(df_long) <= sample_n:
        sample_df = df_long.copy()
    else:
        proportions = df_long[group_col].value_counts(normalize=True)
        raw_counts = proportions * sample_n
        floor_counts = raw_counts.astype(int)
        remainder = raw_counts - floor_counts
        desired = floor_counts.copy()
        remaining = sample_n - desired.sum()
        if remaining > 0:
            for idx in remainder.sort_values(ascending=False).index:
                if remaining <= 0:
                    break
                desired[idx] += 1
                remaining -= 1
        parts = []
        for grp, n in desired.items():
            grp_df = df_long[df_long[group_col] == grp]
            available = len(grp_df)
            if available >= n:
                parts.append(grp_df.sample(n=n, random_state=42))
            else:
                parts.append(grp_df.sample(n=available, random_state=42))
        sample_df = pd.concat(parts)
        if len(sample_df) < sample_n:
            needed = sample_n - len(sample_df)
            remaining_pool = df_long.drop(sample_df.index)
            if len(remaining_pool) >= needed:
                extra = remaining_pool.sample(n=needed, random_state=42)
            else:
                extra = remaining_pool.sample(n=needed, replace=True, random_state=42)
            sample_df = pd.concat([sample_df, extra])
        sample_df = sample_df.sample(frac=1, random_state=42).reset_index(drop=True)
    sample_df.to_csv(out_path, index=False)
    print(f"Saved {name} sample (n={len(sample_df)}) to {out_path}")

# Done

Sampling bill_data_long (source rows: 180000) -> /Users/taylorbaldwin/Downloads/cleaned_bill_amt_sample_1000.csv
Saved bill_data_long sample (n=1000) to /Users/taylorbaldwin/Downloads/cleaned_bill_amt_sample_1000.csv
Sampling pay_status_data_long (source rows: 180000) -> /Users/taylorbaldwin/Downloads/cleaned_pay_status_sample_1000.csv
Saved pay_status_data_long sample (n=1000) to /Users/taylorbaldwin/Downloads/cleaned_pay_status_sample_1000.csv
Sampling pay_amt_data_long (source rows: 180000) -> /Users/taylorbaldwin/Downloads/cleaned_pay_amt_sample_1000.csv
Saved pay_amt_data_long sample (n=1000) to /Users/taylorbaldwin/Downloads/cleaned_pay_amt_sample_1000.csv
Saved pay_amt_data_long sample (n=1000) to /Users/taylorbaldwin/Downloads/cleaned_pay_amt_sample_1000.csv
